### Reproducibility Notebook for: "Non-Hermitian effects in the Su-Schrieffer-Heeger model: Exploring substrate coupling and decoupling dynamics"
### You can access the article at [Phys. Rev. B.](https://journals.aps.org/prb/accepted/10.1103/xjy9-6ysw) (DOI: 10.1103/xjy9-6ysw)
### This notebook provides the data and code required to reproduce the results used to plot the Figure 8 of the above article.
### Dr. Shayan Edalatmanesh

In [1]:
import sisl
from sisl import *
import sisl.viz
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sisl.viz import merge_plots
from sisl.viz.processors.math import normalize
from functools import partial
# To quickly plot the hamiltonian matrix
import plotly.express as px
import cmath
import scipy
from scipy.linalg import ishermitian
from scipy.linalg import schur, eigvals
from matplotlib.colors import LogNorm
import cmocean
import matplotlib.ticker as mticker


In [2]:
cell = sisl.Geometry(
    [[0, 0, 0], [1, 0, 0]],
    [Atom("C", maxR =1.43420), Atom("C", maxR = 1.43420)],
    lattice=Lattice([2.0, 10, 10], nsc=[3, 1, 1]) #, origin=[-5] * 3
)

In [ ]:
N = 10  # number of unit cells
gamma_values = [0.1, 1.0, 2.0, 3.0]
M_values = np.arange(1, 2*N)
energy_min, energy_max, num_E = -2.5, 2.5, 1000
energyGrid = np.linspace(energy_min, energy_max, num_E)

# SSH parameters for the two cases
cases = {
    'Trivial': {'t1': 4/3, 't2': 2/3},
    'Nontrivial': {'t1': 2/3, 't2': 4/3}
}

gamma2 = 1.0  # fixed coupling to the tip

def compute_transmission(t1, t2, gamma1, gamma2, N, M, energyGrid):
    H0 = sisl.Hamiltonian(cell, dtype='complex')
    H0[0, 0] = -1j * gamma1
    H0[1, 1] = -1j * gamma1
    H0[0, 1] = t1
    H0[1, 0] = t1
    H0[0, 1, (-1, 0)] = t2
    H0[1, 0, (1, 0)] = t2
    H = H0.tile(N, 0)
    # Decouple M sites
    for k in range(2*N - M, 2*N):
        H[k, k] = 0.0
    H.set_nsc([1,1,1])
    # Tip coupling on last site
    H[2*N - 1, 2*N - 1] = -1j * gamma2
    Hmat = H.Hk().toarray()

    # Broadening matrices
    gamma_array = np.zeros(2*N)
    gamma_array[:2*N - M] = gamma1
    Gamma1 = 2.0 * np.diag(gamma_array)

    Gamma2 = np.zeros((2*N, 2*N), dtype=complex)
    Gamma2[2*N - 1, 2*N - 1] = 2.0 * gamma2

    # T(E)
    T = np.zeros_like(energyGrid)
    for i, E in enumerate(energyGrid):
        denom = (E) * np.eye(2*N, dtype=complex)- Hmat
        GR   = np.linalg.inv(denom)
        #GR = np.linalg.inv((E * np.eye(2*N)) - Hmat)
        GRdag = GR.conj().T
        Tmat = Gamma1 @ GR @ Gamma2 @ GRdag
        T[i] = np.trace(Tmat).real
    return T


T_data = {
    case_name: [
        np.array([compute_transmission(params['t1'], params['t2'], g, gamma2, N, M, energyGrid)
                  for M in M_values])
        for g in gamma_values
    ]
    for case_name, params in cases.items()
}

# Compute global color scale limits
all_T = np.concatenate([t.flatten() for case in T_data.values() for t in case])
vmin, vmax = 0, 1 #np.max(all_T)

# Plotting
fig, axes = plt.subplots(2, 4, figsize=(25, 10), sharey=True, sharex=False)
fig.subplots_adjust(wspace=0.025, hspace=0.4)  # Adjust spacing

panel_labels = ['b', 'c', 'd', 'e', 'g', 'h', 'i', 'j']
for row, (case_name, t_list) in enumerate(T_data.items()):
    for col, (gamma, T_mat) in enumerate(zip(gamma_values, t_list)):
        ax = axes[row, col]
        # T_mat shape: (len(M_values), len(energyGrid))
        # We want M on x, E on y so → transpose for pcolormesh
        X, Y = np.meshgrid(M_values, energyGrid)
        pcm = ax.pcolormesh(X, Y, T_mat.T, vmin=vmin, vmax=vmax, cmap="turbo", rasterized=True) #, shading='auto')   #viridis #cm.haline
        #pcm = ax.pcolormesh( X, Y, T_mat.T, cmap=cmocean.cm.solar, norm=LogNorm(vmin=1e-3, vmax=vmax), alpha=1.0, shading='auto', rasterized=True) #, shading='auto' )
        #if row == 1:
        ax.set_xlabel('M', fontsize=16)
        if col == 0:
            ax.set_ylabel('E / t', fontsize=14)
        ax.set_title(r'$\gamma/t = {:.1f}$'.format(gamma), fontsize=16)
        ax.text(0.01, 1.1, panel_labels[row*4 + col],
                transform=ax.transAxes, fontsize=16, fontweight='bold', va='top', ha='left')
        ax.set_xticks(M_values)
        labels = [str(m) if m % 2 == 0 else '' for m in M_values]  # '' → no label
        ax.set_xticklabels(labels)
# Colorbar
cbar = fig.colorbar(pcm, ax=axes.ravel().tolist(), pad=0.01)
cbar.set_label('T(E)', fontsize=18)
cbar.ax.tick_params(labelsize=18)

fig.text(0.45, 0.95, "Case 1", ha='center', va='center', fontsize=16) #, fontweight='bold')
fig.text(0.45, 0.49, "Case 2", ha='center', va='center', fontsize=16) #, fontweight='bold')
#pcm.set_rasterized(True)

plt.savefig('figure8.pdf', format='pdf', bbox_inches='tight')
plt.show()
